# Main.py

In [1]:

#import asyncio
from datetime import datetime
from tqdm import tqdm
# # ---------- database ------------
import pandas as pd
#from sqlalchemy import create_engine, URL, text
import psycopg2 as pg
# # ---------- alpar libs ----------
import config 
from data_ops import (
    fetch, 
    latest_workday, 
    db_ops
) 

# from pathlib import Path
import requests 
import more_itertools as mit
# import itertools as it
import sqlite3 as db 


cfg = config.configs 

datetime.strptime(str(latest_workday), '%Y%m%d').strftime('%Y-%m-%d (%A)')

'2023-03-12 (Sunday)'

In [22]:
pd.options.display.max_columns = 40
pd.options.display.max_rows = 5

In [2]:
# -------------extras: settings, sample df------------
# ---------------------settings------------
# 
# with pd.option_context('display.max_colwidth', None,
#                        'display.max_columns', None,
#                        'display.max_rows', None):
#     display(types_df)
#
# -------------sample dummy dataframe-------------------
# 
# prev = pd.DataFrame({
#     'name':['first_name','first_name', 'first_name', 'first_name', 'first_name'],
#     'fetch_date': ['20230122', '20230123', '20230122', '20230123', '20230123'],
#     'id': [100,100,100,125,126]}, 
#     index=['917857106093847', '1050751214677134', '2589887561569709', '3542690854557886', '3772339654185462'])
# update = pd.DataFrame({
#     'name':['second_old_name', 'first_new_name', 'third_old_name_updated', 'second_new_name'],
#     'fetch_date': ['20230123', '20230121', '20230122', '20230123'],
#     'id': [200,200,130,130]}, 
#     index=['1050751214677134', '3542690854456286', '2589887561569709', '3772339112185462'])

# headers = {
#                     'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:101.0) Gecko/20100101 Firefox/101.0',
#                     'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
#                     'Accept-Language': 'en-US,en;q=0.5',
#                     # 'Accept-Encoding': 'gzip, deflate',
#                     'Connection': 'keep-alive',
#                     # 'Cookie': '_ga=GA1.2.1125034676.1645178849',
#                     'Upgrade-Insecure-Requests': '1',
#                     'Pragma': 'no-cache',
#                     'Cache-Control': 'no-cache',
#                 }

# import numpy as np
# dates = pd.date_range('2023-02-01', periods=8)
# df = pd.DataFrame(np.random.randn(8,4), columns=['A','B','C','D'], index=dates)
# df2 = pd.DataFrame(np.random.randn(8,4), columns=['A','B','C','D'], index=dates)

In [30]:
conn = db.connect(cfg['PATH']['DB'])
# region: sqlalchemy connection
# db_url = URL.create(
#     'postgresql+psycopg2',
#     username = cfg['DB']['USERNAME'],
#     password = cfg['DB']['PASSWORD'],
#     host = cfg['DB']['HOST'],
#     database = cfg['DB']['NAME']
# )
# engine = create_engine(db_url, echo=False)
# conn = engine.connect()
# endregion

In [32]:
instruments = pd.read_sql(
    f'''
    SELECT id, tableu_code, mod_date 
    FROM instruments_all
    WHERE mod_date > {str(cfg['LAST_UPDATE']['INSTRUMENTS'])}
    ''', 
    con= conn, 
    index_col= 'id'
)

ids_to_get_attrs_for = instruments[instruments['tableu_code'] != '6'].index 

In [35]:
ids_to_get_attrs_for

Index(['3203898026678865', '10919655792568926', '14427168056799610',
       '16330743198286823', '21727652065573865', '22260326095996531',
       '36773155987365094', '37366608481858080', '40553302624764543',
       '41302553376174581', '42538255636473499', '42892937789646488',
       '47563321799863211', '49869693814643443', '50002340308486819',
       '53686258677793038', '55109311702773448', '60095061789823130',
       '61725225143026283', '62107151670661969', '67612261115225625'],
      dtype='object', name='id')

In [34]:
for chunk in mit.ichunked(ids_to_get_attrs_for, 100):
    print(list(chunk))

['3203898026678865', '10919655792568926', '14427168056799610', '16330743198286823', '21727652065573865', '22260326095996531', '36773155987365094', '37366608481858080', '40553302624764543', '41302553376174581', '42538255636473499', '42892937789646488', '47563321799863211', '49869693814643443', '50002340308486819', '53686258677793038', '55109311702773448', '60095061789823130', '61725225143026283', '62107151670661969', '67612261115225625']


## Valid/Invalid

In [19]:
latest_workday_parsed = datetime.strptime(str(latest_workday), '%Y%m%d')
delta = (latest_workday_parsed.weekday() - 2) % 7
last_wed_date = latest_workday - delta
last_wed_date

20230301

In [ ]:
def load_instruments(*types):
    pass # default=valid, others=invalid, history, saham, preemptive rights, otc

### اینا رو نگه دارم برای بعد ببینم داستانشون چیه. 

In [11]:
insts_overall = fetch.instruments(0)
# آخرین ماد هر نمادی رو ک از این تاریخ تا الان معامله شده باشه رو نشون میده
# بیشتر از این یکی (با افزایش سرمایه) میاره تعداد نمادها رو
fetch.instruments_and_share_increase()
# فکت: ومهان و کیمیا رو ک امروز -یعنی آخرین روز هفته- بعد از بازار بسته شدن رو اصلا توی نمادها نیاورد
# چرا؟ باید ببینم این دو تا چند روز دیگه ک باز شدن چ بلایی سرشون میاد؟ افزایش سرمایه میدن؟ یا چی؟ چون کیمیا ک افزایش سرمایه ثبت کرده. حالا باید ببینم چی میشه...
valid_ziad = insts_overall[insts_overall['mod_date']> '20230301']

In [10]:
insts, share = fetch.instruments_and_share_increase(0,0)
valid_kam = insts[insts['mod_date']> '20230301']

In [12]:
valid_ziad[~valid_ziad.index.isin(valid_kam.index)]

,inst_code_12,inst_code_5,name_latin,company_code_4,ticker,name,isin,mod_date,market_code,company_name,status_code,group_code,market_type_code,tableu_code,industry_sector_code,industry_subsector_code,type_code
id,,,,,,,,,,,,,,,,,
27814844870305607,IRO3KZGZ0001,KZGZ1,Kimia Zanjan Co,KZGZ,کیمیا,معدنی کیمیای زنجان گستران,IRO3KZGZ0008,20230304,2,معدنی کیمیای زنجان گستران,A,Z1,NO,3,27,2720,303
58964593134314938,IRO3MEAZ0001,MEAZ1,Mehr Ayandegan,MEAZ,ومهان,گروه توسعه مالی مهر آیندگان,IRO3MEAZ0007,20230304,2,گروه توسعه مالی مهر آیندگان,A,Z1,NO,3,56,5699,303


### Categorization

In [51]:
insts = pd.read_sql('select * from instruments_aio', conn, index_col='id')
drop_cols = ['inst_code_12','inst_code_5','name_latin','company_code_4',
    'company_name','industry_sector_code','industry_subsector_code',
    'نام شرکت','نام لاتین شرکت','نماد 30 رقمی فارسی','نماد فارسی','کد 12 رقمی شرکت',
    'کد 12 رقمی نماد','کد 4 رقمی شرکت','کد 5 رقمی نماد','کد تابلو']
insts.drop(drop_cols, axis=1, inplace=True)
insts

,ticker,name,isin,mod_date,market_code,status_code,group_code,market_type_code,tableu_code,type_code,بازار,زیر گروه صنعت,کد زیر گروه صنعت,کد گروه صنعت,گروه صنعت,type,sub_type,last_fetch_daily_quotes
id,,,,,,,,,,,,,,,,,,
22129017544200,خاذینح,ح . سایپاآذین,IRR1AZIN0106,20200516,1,A,N2,NO,5,400,بازار دوم بورس,سایر قطعات یدکی و وسایل نقلیه موتوری,3499,34,خودرو و ساخت قطعات,حق تقدم,حق تقدم سهم,20230301
94378630649293,اراد110,مرابحه عام دولت110-ش.خ040401,IRB4SO110443,20221212,2,A,17,NO,4,706,-,اوراق مرابحه دولتی با ضمانت دولت,6945,69,اوراق تامین مالی,اوراق مشارکت,صکوک اختصاصی,20230301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72002976013856737,57-بانکها6,57-بانکها,IRX6X57T0000,20171212,1,I,00,ID,6,68,None,None,None,None,None,شاخص,شاخص,20230301
72044846109864381,پارتا,مجتمع صنعتی آرتاویل تایر,IRO3ARTZ0005,20230304,2,A,Z1,NO,3,303,بازار دوم فرابورس,تولید تایر و بازسازی تایرهای لاستیکی,2511,25,لاستیک و پلاستیک,سهام عادی,اتیسی,20230301


In [61]:
insts['market_code'].unique()

array(['1', '2', '4', '7'], dtype=object)

In [138]:
# List of Tableu Codes
tableu = insts['tableu_code'].unique()
# List of Market Codes
market = insts['market_code'].unique()
# List of Types
types = insts['type_code'].unique()
# Markets
bazar = insts['بازار'].unique()

for code in tableu:
    print(f'Tableu <{code}>:', insts[insts['tableu_code'] == code]['بازار'].unique())

Tableu <5>: ['بازار دوم بورس' 'بازار سوم فرابورس' '-']
Tableu <4>: ['-' 'بازار ابزارهای نوین مالی فرابورس' 'بازار ابزارهای مشتقه فرابورس']
Tableu <1>: ['بازار اول فرابورس' 'بازار اول (تابلوی اصلی) بورس' '-']
Tableu <7>: ['بازار پایه زرد فرابورس' 'بازار پایه نارنجی فرابورس'
 'بازار پایه قرمز فرابورس' '-']
Tableu <3>: ['بازار دوم فرابورس' 'بازار اول (تابلوی فرعی) بورس' 'بازار دوم بورس' '-']
Tableu <9>: ['شرکتهای کوچک و متوسط فرابورس']
Tableu <6>: [None]


#### Filter By: Instrument Type (Saham)

In [173]:
# 'حق تقدم'
# بجز بازار پایه و شرکتهای کوچک و متوسط فرابورس
insts[(insts['type'] == 'حق تقدم') & (insts['tableu_code'] != '9') & (insts['tableu_code'] != '7')]['بازار'].unique()

# سهام عادی
# بجز بازار پایه و شرکتهای کوچک و متوسط فرابورس
saham = insts[
    (insts['type'] == 'سهام عادی') & 
    (insts['tableu_code'] != '9') & 
    (insts['tableu_code'] != '8') & 
    (insts['tableu_code'] != '7') & 
    (insts['tableu_code'] != '4') &
    (insts['بازار'] != '-') 
]
saham

,ticker,name,isin,mod_date,market_code,status_code,group_code,market_type_code,tableu_code,type_code,بازار,زیر گروه صنعت,کد زیر گروه صنعت,کد گروه صنعت,گروه صنعت,type,sub_type,last_fetch_daily_quotes
id,,,,,,,,,,,,,,,,,,
114312662654155,وسپهر,سرمایه گذاری مالی سپهرصادرات,IRO3SAIZ0007,20230304,2,A,1Z,NO,1,303,بازار اول فرابورس,سایر واسطه های مالی,5699,56,سرمایه گذاریها,سهام عادی,اتیسی,20230301
204092872752957,شصدف,صنعتی دوده فام,IRO3SDFZ0007,20230304,2,A,1Z,NO,1,303,بازار اول فرابورس,تولید سایر محصولات شیمیایی,4429,44,محصولات شیمیایی,سهام عادی,اتیسی,20230301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71856634742001725,سپ,پرداخت الکترونیک سامان کیش,IRO1SEPK0003,20230304,1,A,N2,NO,5,300,بازار دوم بورس,مشاوره و تهیه نرم افزار,7220,72,رایانه و فعالیت های وابسته به آن,سهام عادی,سهام,20230301
72044846109864381,پارتا,مجتمع صنعتی آرتاویل تایر,IRO3ARTZ0005,20230304,2,A,Z1,NO,3,303,بازار دوم فرابورس,تولید تایر و بازسازی تایرهای لاستیکی,2511,25,لاستیک و پلاستیک,سهام عادی,اتیسی,20230301


In [179]:
saham[saham['ticker'].str.contains(' ')][['ticker','name','mod_date']]
# 17617474823279712	آ س پ	آ.س.پ
# 27096851668435724	جم پیلن	پلی پروپیلن جم - جم پیلن
# 43362635835198978	های وب	داده گسترعصرنوین-های وب
# 62977319271289925	کی بی سی	شرکت کی بی سی


,ticker,name,mod_date
id,,,
495499165983171,صنعت ساختمان,گسترش فن آوری های صنعت ساختمان,20180721
651564801186566,پیمانکاری نوین,پیمانکاری انرژی نوین,20180721
8187498148443422,شماره گذاری کالا,شماره گذاری کالا و خدمات,20180721
9609565131304189,ایران زمین,پذیره بانک ایران زمین,20180721
11006334882585136,نیروگاه عباسپور,بهره برداری نیروگاه عباسپور,20180721
11452654295102268,مقره سازی,مقره سازی ایران,20180721
16600572108732428,پخش اول,گلپخش اول,20230304
17605403389426895,نیشکر هفت تپه,کشت و صنعت نیشکر هفت تپه,20220108
17617474823279712,آ س پ,آ.س.پ,20230304


In [169]:
insts[
    (insts['sub_type'] == 'اتیسی') & 
    (insts['tableu_code'] != '9') & 
    (insts['tableu_code'] != '8') & 
    (insts['tableu_code'] != '7') & 
    (insts['tableu_code'] != '4') & 
    (insts['بازار'] != '-') 
]['ticker'].nunique()

# عمده - انرژی۲ - پذیره - حذف - انرژی۱ - اونایی ک فاصله دارن(۲ کلمه اند) - بازار=خط تیره منفی
# market 2 & tableu 5 = فرابورس بازار سوم
# 

277

In [ ]:
insts[
    (insts['ticker'].str.contains('حذف')) |
    (insts['ticker'].str.contains('عمده')) |
    (insts['ticker'].str.contains('پذیره')) 
    
]

,ticker,name,isin,mod_date,market_code,status_code,group_code,market_type_code,tableu_code,type_code,بازار,زیر گروه صنعت,کد زیر گروه صنعت,کد گروه صنعت,گروه صنعت,type,sub_type,last_fetch_daily_quotes
id,,,,,,,,,,,,,,,,,,
2432718060936300,وساخت عمده2,س. ساختمان ایران عمده2,IRO1SAKH0205,20230304,1,A,91,NO,1,300,-,سایر فعالیت های املاک و مستغلات,7099,70,انبوه سازی، املاک و مستغلات,سهام عادی,سهام,20230301
13666774088286918,بتهران-پذیره,بیمه اتکایی تهران رواک50%تا-پذ,IRO3Z0790007,20230115,2,A,Z1,NO,5,303,بازار سوم فرابورس,بیمه غیرزندگی,6603,66,بیمه وصندوق بازنشستگی به جزتامین اجتماعی,سهام عادی,اتیسی,20230301
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55626190083788703,وساخت عمده1,س. ساختمان ایران عمده1,IRO1SAKH0106,20230304,1,A,91,NO,1,300,-,سایر فعالیت های املاک و مستغلات,7099,70,انبوه سازی، املاک و مستغلات,سهام عادی,سهام,20230301
55906836177528951,تعاون-پذیره,بیمه تعاون-پذیره,IRO3ZF480008,20220103,2,A,Z1,NO,5,303,بازار سوم فرابورس,فعالیتهای مالی و اجرائی,6611,66,بیمه وصندوق بازنشستگی به جزتامین اجتماعی,سهام عادی,اتیسی,20230301


In [136]:
otc = insts[insts['sub_type'] == 'اتیسی']

In [ ]:
insts[insts['tableu_code'] == '8']

,ticker,tableu_code,market
id,,,
4054414296721,ضلبخند202,8,-
133407505805365,طبدر2033,8,-
183419485240889,ضفگستر1008,8,-
1629946718091595,طسلا7005,8,-
1665565213994526,ضفگستر1010,8,-
...,...,...,...
69843691250564003,ضلبخند508,8,-
70304393734444484,طسلا7001,8,-
70950258269733388,طخود0107,8,-


### Get Trade History (Ticks/Trade Details)

In [3]:
response = req.get(
    f'http://cdn.tsetmc.com/api/Trade/GetTradeHistory/{inst_id}/{desired_date}/false', verify=False, headers=headers)
json_data = json.loads(response.text)
ticks = pd.DataFrame(json_data['tradeHistory']).set_index('nTran').sort_index()
ticks

,insCode,dEven,hEven,qTitTran,pTran,qTitNgJ,iSensVarP,pPhSeaCotJ,pPbSeaCotJ,iAnuTran,xqVarPJDrPRf,canceled
nTran,,,,,,,,,,,,
1,None,0,90037,300000,2941.0,0, ,0.0,0.0,0,0.0,0
2,None,0,90037,100000,2941.0,0, ,0.0,0.0,0,0.0,0
3,None,0,90037,43368,2941.0,0, ,0.0,0.0,0,0.0,0
4,None,0,90037,11960,2941.0,0, ,0.0,0.0,0,0.0,0
5,None,0,90037,10218,2941.0,0, ,0.0,0.0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
6711,None,0,122958,20000,3067.0,0, ,0.0,0.0,0,0.0,0
6712,None,0,122959,20000,3067.0,0, ,0.0,0.0,0,0.0,0
6713,None,0,122959,14250,3067.0,0, ,0.0,0.0,0,0.0,0


In [5]:
ticks['insCode'] = inst_id
ticks['dEven'] = pd.to_datetime(desired_date,format='%Y%m%d')
#ticks['ts'] = pd.to_datetime(ticks['hEven'], format='%H%M%S')
ticks['timestamp']= ticks['dEven'].astype(str) + pd.to_datetime(ticks['hEven'], format='%H%M%S').astype(str)

In [6]:
ticks

,insCode,dEven,hEven,qTitTran,pTran,qTitNgJ,iSensVarP,pPhSeaCotJ,pPbSeaCotJ,iAnuTran,xqVarPJDrPRf,canceled,timestamp
nTran,,,,,,,,,,,,,
1,43362635835198978,2022-08-21,90037,300000,2941.0,0, ,0.0,0.0,0,0.0,0,2022-08-211900-01-01 09:00:37
2,43362635835198978,2022-08-21,90037,100000,2941.0,0, ,0.0,0.0,0,0.0,0,2022-08-211900-01-01 09:00:37
3,43362635835198978,2022-08-21,90037,43368,2941.0,0, ,0.0,0.0,0,0.0,0,2022-08-211900-01-01 09:00:37
4,43362635835198978,2022-08-21,90037,11960,2941.0,0, ,0.0,0.0,0,0.0,0,2022-08-211900-01-01 09:00:37
5,43362635835198978,2022-08-21,90037,10218,2941.0,0, ,0.0,0.0,0,0.0,0,2022-08-211900-01-01 09:00:37
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6711,43362635835198978,2022-08-21,122958,20000,3067.0,0, ,0.0,0.0,0,0.0,0,2022-08-211900-01-01 12:29:58
6712,43362635835198978,2022-08-21,122959,20000,3067.0,0, ,0.0,0.0,0,0.0,0,2022-08-211900-01-01 12:29:59
6713,43362635835198978,2022-08-21,122959,14250,3067.0,0, ,0.0,0.0,0,0.0,0,2022-08-211900-01-01 12:29:59


In [ ]:
ticks.resample()

### Filtering & Ploting

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
pio.templates.default = 'plotly_dark'


In [15]:
# Today's MarketWatch! DataFrame
traded_today_df = pd.DataFrame([data[instrument].iloc[-1] for instrument in instruments_traded_so_far if (data[instrument].index[-1] == last_workday)])


In [6]:
high_value = op.filter_by_value(instrument_tickers=instruments_traded_today, database=data, factor=2, make_df=True)
high_value


,ent code,name,namad,nam,j-date,open,high,low,last,close,...,volume MA 5,volume MA 9,volume MA 3,volume MA 26,volume MA 52,value MA 3,value MA 5,value MA 9,value MA 26,value MA 52
2022-09-10,TB77,TreasuryBill230109-a,اخزا902,اسنادخزانه-م2بودجه99-011019-ت,14010619,937000.0,937000.0,935220.0,936500.0,936910.0,...,28700.0,2.518889e+04,4.353333e+04,2.692308e+04,4.750385e+04,4.077619e+10,2.687331e+10,2.352258e+10,2.494468e+10,4.336457e+10
2022-09-10,TB83,TreasuryBill230828-a,اخزا908,اسنادخزانه-م8بودجه99-020606-ت,14010619,828870.0,829430.0,828000.0,829430.0,829330.0,...,28160.0,2.363333e+04,3.763333e+04,1.716538e+04,4.509423e+04,3.120696e+10,2.332243e+10,1.953626e+10,1.408507e+10,3.614158e+10
2022-09-10,TB91,TreasuryBill230819-a,اخزا916,اسنادخزانه-م16بودجه99-020528-ت,14010619,842180.0,842180.0,842180.0,842180.0,842180.0,...,12409.8,9.119333e+03,1.833533e+04,6.203263e+03,6.203263e+03,1.541292e+10,1.035659e+10,7.508267e+09,4.911234e+09,4.911234e+09
2022-09-10,SALF,AminTadbirFardaETF-a,الماس,صندوق_س.امين_تدبيرگران_فردا-س-ت,14010619,111000.0,114000.0,111000.0,111000.0,112097.0,...,13507.4,1.236522e+04,2.065900e+04,1.207558e+04,1.582867e+04,2.315368e+09,1.513163e+09,1.393643e+09,1.330181e+09,1.777295e+09
2022-09-10,TOSZ,TOSAN-a,توسن,توسعه_سامانه_ي_نرم_افزاري_نگين-ت,14010619,14990.0,15520.0,14550.0,15520.0,15320.0,...,2606217.0,1.957540e+06,3.788956e+06,1.314413e+06,1.201066e+06,5.849467e+10,4.178938e+10,3.372620e+10,2.487222e+10,2.360080e+10
2022-09-10,KSHJ,S*IRI Marine Co.-a,حکشتی,كشتيراني_جمهوري_اسلامي_ايران-ت,14010619,23510.0,23510.0,22560.0,22570.0,22570.0,...,14556392.4,1.709113e+07,1.924198e+07,1.659934e+07,2.091673e+07,4.410619e+11,3.382885e+11,4.153053e+11,3.963030e+11,4.603504e+11
2022-09-10,KKHP,S*Kerman Khodro E.-a,خکرمان,گروه_اقتصادي_كرمان_خودرو-ت,14010619,3648.0,3648.0,3583.0,3648.0,3647.0,...,14766569.0,1.165854e+07,2.159768e+07,1.558829e+07,1.438011e+07,7.805133e+10,5.289414e+10,4.149542e+10,5.428829e+10,4.874815e+10
2022-09-10,INFO,Inf. Services-a,رانفور,خدمات‌انفورماتيك‌-ت,14010619,3930.0,4049.0,3840.0,4049.0,4023.0,...,2740031.4,2.704766e+06,3.020928e+06,3.143687e+06,3.256747e+06,1.205973e+10,1.096140e+10,1.088318e+10,1.251370e+10,1.277916e+10
2022-09-10,ZAKF,Zarin Korosh ETF-a,زرین,صندوق_س._سهام_زرين_كوروش-س-ت,14010619,12000.0,12288.0,11930.0,11950.0,12086.0,...,117864.0,9.022900e+04,1.429957e+05,1.223091e+05,1.599467e+05,1.728267e+09,1.429458e+09,1.098126e+09,1.509757e+09,2.021369e+09
2022-09-10,BTEJ,BTEJ-O-14010720-a,طجار7047,اختيارف_وتجارت-1700-1401/07/20-ت,14010619,200.0,200.0,150.0,175.0,159.0,...,245.0,2.534444e+02,3.613333e+02,2.326500e+02,2.326500e+02,5.696067e+07,3.640440e+07,2.872356e+07,2.106390e+07,2.106390e+07


In [13]:
high_value_plot = px.bar(high_value, 'namad', [
    'value traded', 'value MA 3', 'value MA 5', 'value MA 9', 'value MA 26'], barmode='group')

viz.beautify_chart(high_value_plot)
high_value_plot.show(config=viz.config)


### Fundamentals (Amin)

In [2]:
amin= pd.read_excel('D:/Alpar/assisstant/dash/inputs/amin.xlsx')
amin.set_index('نماد',inplace=True)

In [3]:
fund= pd.DataFrame()
fund['eps ttm'] =amin['قیمت پایانی ریال'] / amin['P/E TTM']
fund['eps f growth'] =amin['قیمت پایانی ریال'] / amin['P/E کارشناسی رشدی']
fund['fiscal year ends on'] =amin['سال مالی منتهی به']
fund['fiscal month'] = amin['آخرین ماه']
fund['activity type'] = amin['نوع فعالیت']
fund['firm'] = amin['شرکت']

In [4]:
fund

,eps ttm,eps f growth,fiscal year ends on,fiscal month,activity type,firm
نماد,,,,,,
فولاد,2411.474666,3779.639390,1401/12/29,M03,تولیدی,فولاد مبارکه اصفهان
فاسمین,3898.692333,8502.915714,1401/12/29,M03,تولیدی,کالسیمین
خراسان,18132.129959,42230.530878,1401/12/29,M03,تولیدی,پتروشیمی خراسان
شپدیس,28070.350333,43444.511091,1401/06/31,M09,تولیدی,پتروشیمی پردیس
شغدیر,15189.028462,22804.475874,1401/12/29,M03,تولیدی,پتروشیمی غدیر
...,...,...,...,...,...,...
مادیرا,138.786268,NaN,1401/12/29,M03,تولیدی,صنایع ماشین های اداری ایران
لپارس,1382.043333,NaN,1401/12/29,M03,تولیدی,کارخانجات پارس الکتریک
کبافق,1247.232099,NaN,1401/12/29,M03,تولیدی,معادن بافق
